This is an experiment with `packetsBeforeAck = 2, 10, 15`.

In [ ]:
import ast
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
database = "home/wmnlab/G/quic_data/"
date = "2024-07-29"
exp_name = "QUIC-ack"
device_name = "MacBookProM1"

exp_file_path_dict = {"ack2": ["", "", "", ""],
                      "ack10": ["", "", "", ""],
                      "ack15": ["", "", "", ""],}

ack2_ul_sent_path = f"/{database}/{date}/{exp_name}/{device_name}/#01/raw/log_20240728_2333_4200_server.csv"
ack2_ul_rcv_path = f"/{database}/{date}/{exp_name}/{device_name}/#01/raw/log_20240728_2333_4200_client.csv"
ack2_dl_sent_path = f"/{database}/{date}/{exp_name}/{device_name}/#01/raw/log_20240728_2333_4201_client.csv"
ack2_dl_rcv_path = f"/{database}/{date}/{exp_name}/{device_name}/#01/raw/log_20240728_2333_4201_server.csv"
exp_file_path_dict["ack2"] = [ack2_ul_sent_path, ack2_ul_rcv_path, ack2_dl_sent_path, ack2_dl_rcv_path]
ack10_ul_sent_path = f"/{database}/{date}/{exp_name}/{device_name}/#02/raw/log_20240728_2337_4200_server.csv"
ack10_ul_rcv_path = f"/{database}/{date}/{exp_name}/{device_name}/#02/raw/log_20240728_2337_4200_client.csv"
ack10_dl_sent_path = f"/{database}/{date}/{exp_name}/{device_name}/#02/raw/log_20240728_2337_4201_client.csv"
ack10_dl_rcv_path = f"/{database}/{date}/{exp_name}/{device_name}/#02/raw/log_20240728_2337_4201_server.csv"
exp_file_path_dict["ack10"] = [ack10_ul_sent_path, ack10_ul_rcv_path, ack10_dl_sent_path, ack10_dl_rcv_path]
ack15_ul_sent_path = f"/{database}/{date}/{exp_name}/{device_name}/#03/raw/log_20240728_2342_4200_server.csv"
ack15_ul_rcv_path = f"/{database}/{date}/{exp_name}/{device_name}/#03/raw/log_20240728_2342_4200_client.csv"
ack15_dl_sent_path = f"/{database}/{date}/{exp_name}/{device_name}/#03/raw/log_20240728_2342_4201_client.csv"
ack15_dl_rcv_path = f"/{database}/{date}/{exp_name}/{device_name}/#03/raw/log_20240728_2342_4201_server.csv"
exp_file_path_dict["ack15"] = [ack15_ul_sent_path, ack15_ul_rcv_path, ack15_dl_sent_path, ack15_dl_rcv_path]

In [ ]:
ack2_ul_sent_df = pd.read_csv(exp_file_path_dict["ack2"][0], sep=',')
ack2_ul_sent_df = ack2_ul_sent_df[(ack2_ul_sent_df['name'] == 'transport:p_sent') & (ack2_ul_sent_df['data'].str.contains("'fr_t': 'ack'"))]

ack10_ul_sent_df = pd.read_csv(exp_file_path_dict["ack10"][0], sep=',')
ack10_ul_sent_df = ack10_ul_sent_df[(ack10_ul_sent_df['name'] == 'transport:p_sent') & (ack10_ul_sent_df['data'].str.contains("'fr_t': 'ack'"))]

ack15_ul_sent_df = pd.read_csv(exp_file_path_dict["ack15"][0], sep=',')
ack15_ul_sent_df = ack15_ul_sent_df[(ack15_ul_sent_df['name'] == 'transport:p_sent') & (ack15_ul_sent_df['data'].str.contains("'fr_t': 'ack'"))]

In [ ]:
ack2_dl_sent_df = pd.read_csv(exp_file_path_dict["ack2"][2], sep=',')
ack2_dl_sent_df = ack2_dl_sent_df[(ack2_dl_sent_df['name'] == 'transport:p_sent') & (ack2_dl_sent_df['data'].str.contains("'fr_t': 'ack'"))]

ack10_dl_sent_df = pd.read_csv(exp_file_path_dict["ack10"][2], sep=',')
ack10_dl_sent_df = ack10_dl_sent_df[(ack10_dl_sent_df['name'] == 'transport:p_sent') & (ack10_dl_sent_df['data'].str.contains("'fr_t': 'ack'"))]

ack15_dl_sent_df = pd.read_csv(exp_file_path_dict["ack15"][2], sep=',')
ack15_dl_sent_df = ack15_dl_sent_df[(ack15_dl_sent_df['name'] == 'transport:p_sent') & (ack15_dl_sent_df['data'].str.contains("'fr_t': 'ack'"))]

In [ ]:
print(len(ack2_ul_sent_df), len(ack10_ul_sent_df), len(ack15_ul_sent_df))
print(len(ack2_dl_sent_df), len(ack10_dl_sent_df), len(ack15_dl_sent_df))

In [ ]:
def getAckInfo(df):
    df = df['data']
    acked_ranges_list = []
    ack_delay_list = []
    ecn_list = []
    # print(acked_ranges_series.iloc[0])
    for i in range(len(df)):
        try: 
            s = df.iloc[i]
            data_dict = json.loads(s.replace("\'", "\""))
            for frame in data_dict['frames']:
                if 'acked_ranges' in frame:
                    for range_entry in frame['acked_ranges']:
                        acked_ranges = range_entry
            for frame in data_dict['frames']:
                if 'ack_delay' in frame:
                    ack_delay = frame['ack_delay']
                    ack_delay_list.append(ack_delay)
                if 'ce' in frame:
                    if frame['ce'] != 0:
                        ecn_list.append(True)
                    else:
                        ecn_list.append(False)
            # Extract 'acked_ranges' from all frames
            acked_ranges = [range_entry for frame in data_dict['frames'] if 'acked_ranges' in frame for range_entry in frame['acked_ranges']]
            acked_ranges_list.append(acked_ranges)
            # TODO: add ack_delay & ecn
            
        except:
            print(s, i)
            break

    acked_ranges_df = pd.DataFrame({"acked_ranges": acked_ranges_list})
    ack_delay_df = pd.DataFrame({"ack_delay": ack_delay_list})
    ecn_df = pd.DataFrame({"ecn": ecn_list})

    ack_info_df = pd.concat([acked_ranges_df, ack_delay_df, ecn_df], axis=1)
    ack_info_df = ack_info_df.reset_index(drop=True)

    return ack_info_df

In [ ]:
ack2_dl_ack_info_df = getAckInfo(ack2_dl_sent_df)
ack2_dl_ack_info_df = ack2_dl_ack_info_df.reset_index(drop=True)
ack2_dl_sent_df = ack2_dl_sent_df.reset_index(drop=True)
ack2_dl_sent_df = pd.concat([ack2_dl_sent_df, ack2_dl_ack_info_df], axis=1)
ack2_dl_sent_df = ack2_dl_sent_df.reset_index(drop=True)

ack2_ul_ack_info_df = getAckInfo(ack2_ul_sent_df)
ack2_ul_ack_info_df = ack2_ul_ack_info_df.reset_index(drop=True)
ack2_ul_sent_df = ack2_ul_sent_df.reset_index(drop=True)
ack2_ul_sent_df = pd.concat([ack2_ul_sent_df, ack2_ul_ack_info_df], axis=1)
ack2_ul_sent_df = ack2_ul_sent_df.reset_index(drop=True)

ack10_dl_ack_info_df = getAckInfo(ack10_dl_sent_df)
ack10_dl_ack_info_df = ack10_dl_ack_info_df.reset_index(drop=True)
ack10_dl_sent_df = ack10_dl_sent_df.reset_index(drop=True)
ack10_dl_sent_df = pd.concat([ack10_dl_sent_df, ack10_dl_ack_info_df], axis=1)
ack10_dl_sent_df = ack10_dl_sent_df.reset_index(drop=True)

ack10_ul_ack_info_df = getAckInfo(ack10_ul_sent_df)
ack10_ul_ack_info_df = ack10_ul_ack_info_df.reset_index(drop=True)
ack10_ul_sent_df = ack10_ul_sent_df.reset_index(drop=True)
ack10_ul_sent_df = pd.concat([ack10_ul_sent_df, ack10_ul_ack_info_df], axis=1)
ack10_ul_sent_df = ack10_ul_sent_df.reset_index(drop=True)

ack15_dl_ack_info_df = getAckInfo(ack15_dl_sent_df)
ack15_dl_ack_info_df = ack15_dl_ack_info_df.reset_index(drop=True)
ack15_dl_sent_df = ack15_dl_sent_df.reset_index(drop=True)
ack15_dl_sent_df = pd.concat([ack15_dl_sent_df, ack15_dl_ack_info_df], axis=1)
ack15_dl_sent_df = ack15_dl_sent_df.reset_index(drop=True)

ack15_ul_ack_info_df = getAckInfo(ack15_ul_sent_df)
ack15_ul_ack_info_df = ack15_ul_ack_info_df.reset_index(drop=True)
ack15_ul_sent_df = ack15_ul_sent_df.reset_index(drop=True)
ack15_ul_sent_df = pd.concat([ack15_ul_sent_df, ack15_ul_ack_info_df], axis=1)
ack15_ul_sent_df = ack15_ul_sent_df.reset_index(drop=True)

In [ ]:
def plotAckedRanges(df):
    plt.figure(figsize=(12, 6))
    # Iterate over rows in the DataFrame
    for index, row in df.iterrows():
        ranges = row['acked_ranges']

        # If ranges is a list (and not empty), plot vertical lines
        if isinstance(ranges, list):
            for value in ranges:
                if len(value) < 2:
                    plt.plot([index, index], [value[0], value[0]], color='b', linestyle='-', alpha=0.7)
                else:
                    plt.plot([index, index], [value[0], value[1]], color='b', linestyle='-', alpha=0.7)

    # Labels and title
    plt.xlabel('Row Index')
    plt.ylabel('Acked Ranges')
    plt.title('Vertical Lines Representing Acked Ranges')
    
    plt.grid(True)
    plt.show()


In [ ]:
# plotAckedRanges(ack2_dl_sent_df)
# plotAckedRanges(ack2_ul_sent_df)
# plotAckedRanges(ack10_dl_sent_df)
# plotAckedRanges(ack10_ul_sent_df)
# plotAckedRanges(ack15_dl_sent_df)
# plotAckedRanges(ack15_ul_sent_df)

In [ ]:
mean_ack2_dl_delay = ack2_dl_sent_df['ack_delay'].mean()
mean_ack2_ul_delay = ack2_ul_sent_df['ack_delay'].mean()
print(mean_ack2_dl_delay)
print(mean_ack2_ul_delay)
ack2_dl_sent_df.iloc[100000:100010]['acked_ranges']
ack2_ul_sent_df.iloc[100000:100010]['acked_ranges']

In [ ]:
mean_ack10_dl_delay = ack10_dl_sent_df['ack_delay'].mean()
mean_ack10_ul_delay = ack10_ul_sent_df['ack_delay'].mean()
print(mean_ack10_dl_delay)
print(mean_ack10_ul_delay)
ack10_dl_sent_df.iloc[100000:100010]['acked_ranges']
print(mean_ack10_ul_delay)
ack10_ul_sent_df.iloc[100000:100010]['acked_ranges']

In [ ]:
mean_ack15_dl_delay = ack15_dl_sent_df['ack_delay'].mean()
mean_ack15_ul_delay = ack15_ul_sent_df['ack_delay'].mean()
print(mean_ack15_dl_delay)
print(ack15_dl_sent_df.iloc[100000:100010]['acked_ranges'])
print(mean_ack15_ul_delay)
print(ack15_ul_sent_df.iloc[100000:100010]['acked_ranges'])